In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

from tensorflow.python.keras.models import load_model

In [2]:
K.clear_session()

model=tf.keras.models.load_model('./model_resnet50_100.h5')
tc=tf.lite.TFLiteConverter.from_keras_model(model)

tf.__version__

'2.3.0'

In [3]:
import cv2 as cv
import numpy as np
import os, glob, time

w = 224
h = 224
c = 3

train_path='./train'
val_path='./test'

In [4]:
def read_image(path):
    label_dir = [path+'\\'+x for x in os.listdir(path) if os.path.isdir(path+'\\'+x)]
    files=[]
    labels=[]
    for index,folder in enumerate(label_dir):
        print(index,folder)
        for imgpath in glob.glob(folder+'\\*.*'):
            files.append(imgpath)
            labels.append(index)
        print('%s --> Load Iamge:%d'%(path,len(labels)))
        files=np.asarray(files,dtype=np.str)
        labels=np.asarray(labels,dtype=np.int)
        num=labels.shape[0]
        return files,labels,num


In [5]:
train_images,train_labels,train_image_num = read_image(train_path)
val_images,val_labels,val_image_num = read_image(val_path)

0 ./train\left
./train --> Load Iamge:364
0 ./test\left
./test --> Load Iamge:191


c:\users\andy\anaconda3\envs\robot_tflite\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # This is added back by InteractiveShellApp.init_path()
c:\users\andy\anaconda3\envs\robot_tflite\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more 

In [6]:
def representative_dataset_gen():
    global train_images
    images=train_images
    file_index = np.arange(images.shape[0])
    np.random.shuffle(file_index)
    file_shuffe = images[file_index]
    for file_name in file_shuffe[0:5000]:
        img=cv.imread(file_name)
        img=img.astype(np.float32)/255.0
        img=cv.resize(img,(w,h))
        img=cv.cvtColor(img,cv.COLOR_BGR2RGB)
        img=img.reshape((w,h,c))
        img=np.expand_dims(img,0)
        yield [img]

In [7]:
tc.optimizations=[tf.lite.Optimize.DEFAULT]
tc.target_spec.supported_ops=[tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tc.inference_input_type=tf.uint8
tc.inference_output_type=tf.uint8
tc.representative_dataset=representative_dataset_gen
tflite=tc.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\andy\AppData\Local\Temp\tmpelyorn5q\assets


In [8]:
with tf.io.gfile.GFile('./model_resnet50.tflite','wb') as f:
    f.write(tflite)
print('OK!!!')

OK!!!
